比如对于 MNIST 数字，全局模式就是涉及所有像素的模式，这个参数太多了，随便进行一次反向传播计算量都是巨大的，从计算资源和调参的角度都不建议用传统的神经网络，参数过多还会导致出现过拟合情况），而卷积层学到的是局部模式，也就是参数会少很多！！！！那什么是卷积核呢

<div  align="center">    
    <img src = './img/16.jpg', width=600, height=200/>
    <img src = './img/17.jpg', width=600, height=200/>
    <img src = './img/18.jpg', width=600, height=200/>
</div>

In [1]:
# 现在我们来验证一下
from keras import layers 
from keras import models
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.summary()

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
Total params: 320
Trainable params: 320
Non-trainable params: 0
_________________________________________________________________


确实是（26, 26, 32）

最大池化

<div  align="center">    
    <img src = './img/19.jpg', width=600, height=200/>
</div>

In [2]:
# 现在我们来验证一下
from keras import layers 
from keras import models
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu')) 
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))  # 多分类用softmax， 二分类可以用sigmoid
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

在次模型的基础上训练一下我们的手写收据集

In [3]:
import numpy as np
import matplotlib.pyplot as plt
# 加载数据集
path = './files/mnist.npz'
f = np.load(path)
train_x, train_y = f['x_train'], f['y_train']
test_x, test_y = f['x_test'], f['y_test']
f.close()

In [17]:
test_x.shape

(10000, 28, 28, 1)

In [15]:
train_y.shape

(60000, 10, 2)

In [18]:
# 因为我们的模型输入需要是3D，所以对我们数据的维度进行reshape一下
train_x = train_x.reshape((60000, 28, 28, 1))
train_x = train_x.astype('float64') / 255    # 相当于进行数据的归一化，因为像素是0-255之间的
test_x = test_x.reshape((10000, 28, 28, 1))
test_x = test_x.astype('float64') / 255
from keras.utils import to_categorical
train_y = to_categorical(train_y)  # 化成独热码的形式[1, 0, 0, 0, 0, 0, 0, 0, 0, 0] 代表的是1
test_y = to_categorical(test_y)    #                 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0] 代表的是2

train_y.shape

(60000, 10, 2, 2)

In [ ]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_x, train_y, epochs=5, batch_size=64)

END